In [1]:
import pandas as pd
import pickle


# Calculating Nearest Neighbor blocks & associated stats

### Address data from .ipnyb
* import data, group block and average x,y coordinates

In [2]:
with open("df_address.pkl", 'rb') as picklefile: 
    df4 = pickle.load(picklefile)

In [3]:
df = df4[['Block','XCoord','YCoord']].copy()
df['x'] = 1

In [4]:
df = df.groupby(['Block','XCoord','YCoord'], as_index=False).x.count()
del df['x']

In [5]:
df = df.groupby('Block')['XCoord','YCoord'].mean()

###  Calculating Neareast Neighbors
* create distance matrix
* iterate through nearest neighbors (smallest = number of neighbors + 1)
* output dataframe (df_closest) with closest neighbors of each block (in order!)

In [6]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(df.values, metric='euclidean')
dist_matrix = squareform(distances)

In [7]:
indicies = ['block' + str(i) for i in df.index.tolist()]

In [8]:
dist_matrix = pd.DataFrame(dist_matrix, columns = indicies, index = df.index)

In [9]:
smallest = 6

m_len = len(dist_matrix.columns)
for col in dist_matrix.columns[:m_len]:
    l = dist_matrix[col].nsmallest(smallest).index
    for i in range(len(l)):
        col_2c = m_len + i
        row_2c = int(col[5:])
        dist_matrix.loc[row_2c, col_2c] = int(l[i])
        
k = 0
for j in range(m_len, m_len + smallest):
    k += 1
    dist_matrix.rename(columns={j: 'closest_' + str(k-1)}, inplace=True)

In [10]:
df_closest = dist_matrix.iloc[:,-smallest:].reset_index()

### Pickle it for use later

In [11]:
with open('df_closest.pkl', 'wb') as picklefile:
    pickle.dump(df_closest, picklefile)

### Get last 4 quarter information for each block 
* Combine this w/ df_closest to get the last 4 quarter information for closest neighbors
* This is some crazy join logic - we've got to re-join it for each nearest neighbor
* Only keep necessary columns

In [12]:
# comes from calculating_quarter_data.ipynb
with open("df_l4.pkl", 'rb') as picklefile: 
    df_l4 = pickle.load(picklefile)

In [13]:
for i in range(1,smallest):
    df_closest['closest_' + str(i)] = df_closest['closest_' + str(i)].astype(int)

In [14]:
df9 = df_l4.merge(df_closest, left_on='BLOCK', right_on='Block')
del df9['Block']

In [15]:
for i in range(1,smallest):
    df9 = df9.merge(df_l4, left_on=['INSPECTION_YEAR_Q','closest_' + str(i)], right_on=['INSPECTION_YEAR_Q','BLOCK'])
    del df9['BLOCK_y']
    df9.rename(columns={'last_4_ins_y': 'c' + str(i) + '_l4i', 'last_4_rats_y': 'c'+str(i)+'_l4r','BLOCK_x': 'BLOCK',
                       'last_4_ins_x': 'last_4_ins','last_4_rats_x': 'last_4_rats'}, inplace=True)

In [16]:
col_to_keep = ['INSPECTION_YEAR_Q', 'BLOCK', 'last_4_ins', 'last_4_rats', 
               'c1_l4i', 'c1_l4r', 'c2_l4i', 'c2_l4r', 'c3_l4i', 'c3_l4r', 'c4_l4i', 'c4_l4r', 
               'c5_l4i', 'c5_l4r']
df9 = df9[col_to_keep]

### Pickle it for later use in modeling

In [17]:
with open('last_4_data.pkl', 'wb') as picklefile:
    pickle.dump(df9, picklefile)